Задача 5. Подсчет количества ретвитов на твиты пользователей

Напишите программу, которая подсчитывает количество ретвитов на твиты каждого пользователя в течение 1 мин. с накоплением. Выведите результат в отсортированном по убыванию виде: id твита, количество ретвитов. Каждую 1 мин. сохранять результат в файл. Накапливать не менее 30 мин. После этого для 5 наиболее популярных твитов вывести: screen_name пользователя, id твита и его текст.

Исходные данные

id пользователей: "285532415", "147964447", "34200559", "338960856", "200036850", "72525490", "20510157", "99918629"

In [ ]:
source /opt/rh/rh-python36/enable 
sudo service kafka-server start

In [ ]:
# -*- coding: utf-8 -*-

import tweepy
from tweepy.streaming import json
from kafka import KafkaProducer


producer = KafkaProducer(bootstrap_servers="localhost:9092")
topic_name = "retweets-kafka"

watched_users = ["285532415", "147964447", "34200559", "338960856", 
    "200036850", "72525490", "20510157", "99918629", "82299300", "1070777285924569089"]

class MyStreamListener(tweepy.StreamListener):
    
    def on_data(self, raw_data):

        data = json.loads(raw_data)

        retweeted_status = data.get('retweeted_status')
        if retweeted_status is not None:
            author = retweeted_status.get('user')
            if author is None:
                return

            author_name = author.get('name')
            tweet_id = retweeted_status.get('id')
            text = retweeted_status.get('text')

            if author_name is None or tweet_id is None or text is None:
                return

            result = dict()
            result['author_name'] = author_name
            result['tweet_id'] = tweet_id
            result['text'] = text

            print(result)

            producer.send(topic_name, str(result).encode("utf-8"))

    def on_error(self, status):
        print(status)

consumer_token = "top secret"
consumer_secret = "top secret" 
access_token = "top secret-top secret"
access_secret = "top secret"

auth = tweepy.OAuthHandler(consumer_token, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)

myStreamListener = MyStreamListener()

myStream = tweepy.Stream(auth=api.auth, listener=myStreamListener)

myStream.filter(follow=watched_users)

In [ ]:
# -*- coding: utf-8 -*-
import sys

from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

zk_server = "localhost:2181" # сервер Zookeeper
topic = "retweets-kafka" # топик

# Функция для обновления значений количества слов
def updateTotalCount(currentCount, countState):
    if countState is None:
        countState = 0
    return sum(currentCount, countState)

# Создаем Spark Context
sc = SparkContext(appName="KafkaTwitterWordCount")

sc.setLogLevel("OFF")

# Создаем Streaming Context
ssc = StreamingContext(sc, 60)

# Объявляем checkpoint и указываем директорию в HDFS, где будут храниться значения
ssc.checkpoint("tmp_spark_streaming1")

# Создаем подписчика на поток от Kafka c топиком topic = "tweets-kafka"
kafka_stream = KafkaUtils.createStream(ssc, zk_server, "spark-streaming-consumer", {topic: 1})

# Трансформируем мини-batch 
retweets_stream = kafka_stream.map(lambda x: (x[1], 1)).reduceByKey(lambda x1, x2: x1 + x2)

retweets_total_count = retweets_stream.updateStateByKey(updateTotalCount)

top_retweets = retweets_total_count.transform(lambda rdd: rdd.sortBy(lambda x: x[1], ascending=False))
id_count_tweet = top_retweets.map(lambda x: (eval(x[0])['tweet_id'], x[1]))

id_count_tweet.pprint(5)
# id_count_tweet.transform(lambda rdd: rdd.coalesce(1)).saveAsTextFiles("file:///home/cloudera/712/5_task/output/output")

windowed_data = top_retweets.reduceByKeyAndWindow(max, None, windowDuration=60 * 10, slideDuration=60 * 10)
windowed_data.map(lambda x: x[0]).pprint(5)

# Запускаем Spark Streaming
ssc.start()

# Ожидаем остановку
ssc.awaitTermination()

In [ ]:
-------------------------------------------
Time: 2018-12-20 07:53:00
-------------------------------------------
(1075779880988495874, 3)
(1075712448181780480, 3)
(1075777405359255553, 3)
(1075699832285736960, 3)
(1075762690646102016, 3)
...

-------------------------------------------
Time: 2018-12-20 07:54:00
-------------------------------------------
(1075779880988495874, 4)
(1075699832285736960, 4)
(1075670731755806720, 4)
(1075712448181780480, 3)
(1075770316884824064, 3)
...

-------------------------------------------
Time: 2018-12-20 07:55:00
-------------------------------------------
(1075712448181780480, 5)
(1075779880988495874, 4)
(1075699832285736960, 4)
(1075762690646102016, 4)
(1075670731755806720, 4)
...

-------------------------------------------
Time: 2018-12-20 07:56:00
-------------------------------------------
(1075699832285736960, 7)
(1075779880988495874, 5)
(1075712448181780480, 5)
(1075670731755806720, 5)
(1075762690646102016, 4)
...

-------------------------------------------
Time: 2018-12-20 07:57:00
-------------------------------------------
(1075699832285736960, 9)
(1075670731755806720, 6)
(1075779880988495874, 5)
(1075712448181780480, 5)
(1075762690646102016, 5)
...

-------------------------------------------
Time: 2018-12-20 07:58:00
-------------------------------------------
(1075699832285736960, 11)
(1075762690646102016, 6)
(1075670731755806720, 6)
(1075779880988495874, 5)
(1075712448181780480, 5)
...

-------------------------------------------
Time: 2018-12-20 07:59:00
-------------------------------------------
(1075699832285736960, 13)
(1075712448181780480, 9)
(1075762690646102016, 6)
(1075670731755806720, 6)
(1075779880988495874, 5)
...

-------------------------------------------
Time: 2018-12-20 08:00:00
-------------------------------------------
(1075699832285736960, 19)
(1075712448181780480, 9)
(1075762690646102016, 8)
(1075670731755806720, 6)
(1075779880988495874, 5)
...

-------------------------------------------
Time: 2018-12-20 08:01:00
-------------------------------------------
(1075699832285736960, 21)
(1075712448181780480, 11)
(1075670731755806720, 9)
(1075762690646102016, 8)
(1075707031707926528, 6)
...

-------------------------------------------
Time: 2018-12-20 08:02:00
-------------------------------------------
(1075699832285736960, 23)
(1075712448181780480, 11)
(1075670731755806720, 9)
(1075762690646102016, 8)
(1075770316884824064, 7)
...

-------------------------------------------
Time: 2018-12-20 08:02:00
-------------------------------------------
{'author_name': 'РИА Новости', 'tweet_id': 1075779880988495874, 'text': 'Мадрид обвинил Пучдемона в растрате восьми миллионов евро на референдум \nhttps://t.co/XUKsKRIrft https://t.co/2YxhmdScUf'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075712448181780480, 'text': 'Самый смешной момент. Британский подданный Брилёв от имени российского телевидения спрашивает Путина об отношениях с Британией.'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075770316884824064, 'text': 'После прямой линии Путина о\xa0чем будем говорить на\xa0моей прямой линии? https://t.co/og2DuO69xu'}
{'author_name': 'Сталингулаг', 'tweet_id': 1075720392663281664, 'text': '— Владимир Владимирович, мы стали беднее, цены растут, денег не хватает...\n— На самом деле всё не так, вот слушай,… https://t.co/XfdIGKf7ht'}
{'author_name': 'Фёдор Крашенинников', 'tweet_id': 1075718978847293440, 'text': 'Путин бесконечно готов рассуждать о проблемах коренных парижан.'}
...


{'author_name': 'РЕН ТВ | Новости', 'tweet_id': 1075777405359255553, 'text': 'В Херсоне чиновник избил пенсионера и женщину в рабочем кабинете — #ВИДЕО https://t.co/IPtCRuz5zF https://t.co/0oSgnlN6HT'}
{'author_name': 'НТВ', 'tweet_id': 1075776196384763904, 'text': 'В интервью 17-летней Регине Путин признался, что не воспринимает тяжелую музыку. «Heavy metal для меня очень сложно… https://t.co/s1s9QIk1ro'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075691448765427713, 'text': 'Путина спрашивают про общественные, культурные и прочие советы. Он начинает описывать подвиг десантников. Зал хлопает.\nПаноптикум какой-то.'}
{'author_name': 'РЕН ТВ | Новости', 'tweet_id': 1075770497474805760, 'text': '"Тут не мойка, тварь": охранник закрыл дверь в травмпункт перед окровавленными девушками https://t.co/3g6fcFBlpO https://t.co/uXQK54zSR3'}
{'author_name': 'настоящий Низовцев', 'tweet_id': 1075680749305630720, 'text': 'А вот и я https://t.co/pXJgcvRlQn'}
{'author_name': 'ТАСС', 'tweet_id': 1075777338854432777, 'text': 'Де Мистура назвал конституционный комитет не единственной базой для урегулирования в Сирии:\nhttps://t.co/Jr247JpUrA https://t.co/wUYS5rDo6U'}
{'author_name': 'НТВ', 'tweet_id': 1075747767396524033, 'text': 'Давно что-то в Раде не было мордобоя. И вот. Депутаты передрались после принятия закона, обязывающего УПЦ указать в… https://t.co/zpckujDYnh'}
{'author_name': 'Навальный LIVE', 'tweet_id': 1075690757703458816, 'text': 'Конфликт между Чечней и Ингушетией — будет ли Путин говорить об этом в сегодняшней пресс-конференции? Вопрос Путину… https://t.co/5fPrplyeok'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075770316884824064, 'text': 'После прямой линии Путина о\xa0чем будем говорить на\xa0моей прямой линии? https://t.co/og2DuO69xu'}
{'author_name': 'РИА Новости', 'tweet_id': 1075777490428223489, 'text': 'Казанский авиазавод начал производство первых модернизированных Ту-160 \nhttps://t.co/BPZNcUGFVX https://t.co/FZCAOyYo8w'}
{'author_name': 'RT на русском', 'tweet_id': 1075736675316326400, 'text': 'Иностранный телеканал задал Путину вопрос, почему в Крым поставляют большое количество вооружения.\n\nПрезидент в сот… https://t.co/L677mFUiPj'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075684302313189376, 'text': 'Лучшая прямая трансляция пресс-конференции Путина (с комментариями!) здесь: https://t.co/Mp4iObvBzE'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075712448181780480, 'text': 'Самый смешной момент. Британский подданный Брилёв от имени российского телевидения спрашивает Путина об отношениях с Британией.'}
{'author_name': 'Навальный LIVE', 'tweet_id': 1075662493807079425, 'text': 'Сегодня Владимир Путин проводит свою очередную пресс-конференцию. Мы спросили у людей на улицах, какие вопросы они… https://t.co/5CtJ2pPvAd'}
{'author_name': 'РИА Новости', 'tweet_id': 1075779880988495874, 'text': 'Мадрид обвинил Пучдемона в растрате восьми миллионов евро на референдум \nhttps://t.co/XUKsKRIrft https://t.co/2YxhmdScUf'}
{'author_name': 'НТВ', 'tweet_id': 1075770598880555008, 'text': 'Путин снова исполнил мечту тяжелобольного ребенка. После большой пресс-конференции он побеседовал с 17-летней Регин… https://t.co/UC648J8OPy'}
{'author_name': 'Навальный LIVE', 'tweet_id': 1075681995064111104, 'text': 'Путин подводит итоги года: говорит, доходы и средние зарплаты россиян растут. Тем не менее, по данным Росстата, дин… https://t.co/vjtPcs60fe'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075770316884824064, 'text': 'После прямой линии Путина о\xa0чем будем говорить на\xa0моей прямой линии? https://t.co/og2DuO69xu'}
{'author_name': 'ТАСС', 'tweet_id': 1075686046145396736, 'text': 'Президент Польши принял решение отозвать посла страны на Украине:\nhttps://t.co/pGYQwUTnGM https://t.co/KlNByfngGY'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075661223750979585, 'text': 'Через час начнется пресс-конференция Путина и он будет очень много врать. Об экономических успехах, о демографии, о… https://t.co/OjZcxSLbYc'}
{'author_name': 'НТВ', 'tweet_id': 1075730751906344961, 'text': '— Как вы себя чувствуете?\n— Не дождетесь.\n#прессконференция https://t.co/U7HaeSKXau'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075657579605487617, 'text': 'Требуем допустить Ромуальда на прессуху Путина! https://t.co/xaN1ZNcGR7'}
{'author_name': 'ТАСС', 'tweet_id': 1075679094975524864, 'text': '#Прямосейчас смотрите 14-ю большую пресс-конференцию Владимира Путина. Видео: ТАСС/Ruptly https://t.co/zYEVMLxYTC'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075655976139874304, 'text': 'Шок-контент: доходы и квартиры оппозиционеров. С документами https://t.co/LNU27i8hMC'}
{'author_name': 'ТАСС', 'tweet_id': 1075770371813457921, 'text': 'Пробы с просверленного "Союза МС-09" передадут следствию 24 декабря, сообщил источник:\nhttps://t.co/pEfrLsVI2x\n\n© Г… https://t.co/QC3B9s3Kbg'}
{'author_name': 'РИА Новости', 'tweet_id': 1075759313338163200, 'text': 'Дед Мороз и Йоулупукки встретились на границе России и Финляндии \nhttps://t.co/MCbdrrNxrk https://t.co/ztLMwuQmu8'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075661223750979585, 'text': 'Через час начнется пресс-конференция Путина и он будет очень много врать. Об экономических успехах, о демографии, о… https://t.co/OjZcxSLbYc'}
{'author_name': 'РИА Новости', 'tweet_id': 1075779880988495874, 'text': 'Мадрид обвинил Пучдемона в растрате восьми миллионов евро на референдум \nhttps://t.co/XUKsKRIrft https://t.co/2YxhmdScUf'}
{'author_name': 'Навальный LIVE', 'tweet_id': 1075650114960519169, 'text': 'Если вы правда хотите смотреть, как Путин снова всем чего-то обещает, то лучше делайте это вместе с нами. Будем вес… https://t.co/ilpERMHjin'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075699832285736960, 'text': 'Реально потрясающе. Путина спрашивают про свалки, а он: а вы про острова мусора в Тихом океане знаете?\n"Вы что, хот… https://t.co/MUsCpXCWQW'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075707031707926528, 'text': 'На этой скучной, тоскливой пресс-конференции Путина точно есть как минимум один хороший журналист https://t.co/6CzQlERizA'}
{'author_name': 'Сталингулаг', 'tweet_id': 1075354436032180226, 'text': '«Правительство Франции по традиции раздаст живущим за чертой бедности полмиллиарда евро на Рождество»\n\nА в России д… https://t.co/xBNJQOz5xk'}
{'author_name': 'Сталингулаг', 'tweet_id': 1075720392663281664, 'text': '— Владимир Владимирович, мы стали беднее, цены растут, денег не хватает...\n— На самом деле всё не так, вот слушай,… https://t.co/XfdIGKf7ht'}
{'author_name': 'НТВ', 'tweet_id': 1075670731755806720, 'text': '71-летняя хабаровская пенсионерка принесла в полицию сверток с 250 тыс. рублей, который она нашла по дороге в магаз… https://t.co/Fln64Ejeqq'}
{'author_name': 'Фёдор Крашенинников', 'tweet_id': 1075718978847293440, 'text': 'Путин бесконечно готов рассуждать о проблемах коренных парижан.'}
{'author_name': 'Vitaly Kolesnikov', 'tweet_id': 1075699468362747904, 'text': '— Владимир Владимирович, а что со свалками делать будет?\n—Вы знаете, в тихом океане есть целые острова мусора...'}
{'author_name': 'РИА Новости', 'tweet_id': 1075774282569904130, 'text': 'Посол Польши на Украине покинет пост в конце января \nhttps://t.co/JWBj4aFxUF https://t.co/rldd5sAfWI'}
{'author_name': 'Проф. Преображенский', 'tweet_id': 1075717989180338176, 'text': 'Журналист:\n- Давайте отменим повышение пенсионного возраста!\nПутин: \n- Нет, сейчас страна развивается, крепнет, поэ… https://t.co/DjRHtml6nQ'}
{'author_name': 'Интерфакс', 'tweet_id': 1075781043171590144, 'text': 'Факультет журналистики МГУ объявил лауреата стипендии "Интерфакса-2019" https://t.co/nw4HWySf32'}
{'author_name': 'ТАСС', 'tweet_id': 1075775659052478465, 'text': 'Владимир Путин убежден в правильности и неизбежности увеличения пенсионного возраста. Глава государства добавил, чт… https://t.co/ZeIFhuNd2S'}
{'author_name': 'Интерфакс', 'tweet_id': 1075776781993500674, 'text': 'Украинские корабли снова собрались пройти через Керченский пролив https://t.co/0VuIMYqpBd'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075712448181780480, 'text': 'Самый смешной момент. Британский подданный Брилёв от имени российского телевидения спрашивает Путина об отношениях с Британией.'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075707031707926528, 'text': 'На этой скучной, тоскливой пресс-конференции Путина точно есть как минимум один хороший журналист https://t.co/6CzQlERizA'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075712448181780480, 'text': 'Самый смешной момент. Британский подданный Брилёв от имени российского телевидения спрашивает Путина об отношениях с Британией.'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075762690646102016, 'text': 'Очень круто. Помните такого жулика Серуканова -"очередного разочаровавшегося волонтёра Навального"? СМИ ещё обожали… https://t.co/vzCCVUgMO6'}
{'author_name': 'Первый канал', 'tweet_id': 1075671411400863744, 'text': 'https://t.co/Bxvx3h2cmP'}
{'author_name': 'РИА Новости', 'tweet_id': 1075764980778057730, 'text': 'В Грузии ответили на заявления России о лаборатории Лугара \nhttps://t.co/PODvxlUpHM https://t.co/MnExftacbP'}
{'author_name': 'РЕН ТВ | Новости', 'tweet_id': 1075770497474805760, 'text': '"Тут не мойка, тварь": охранник закрыл дверь в травмпункт перед окровавленными девушками https://t.co/3g6fcFBlpO https://t.co/uXQK54zSR3'}
{'author_name': 'Интерфакс', 'tweet_id': 1075764009532444678, 'text': 'Пьяные украинцы пытались прорваться в Белоруссию https://t.co/RAeApkCjYH'}
{'author_name': 'РИА Новости', 'tweet_id': 1075777490428223489, 'text': 'Казанский авиазавод начал производство первых модернизированных Ту-160 \nhttps://t.co/BPZNcUGFVX https://t.co/FZCAOyYo8w'}
{'author_name': 'Интерфакс', 'tweet_id': 1075724075799068673, 'text': 'Президент пообещал не допустить существенного роста цен на бензин в 2019 году https://t.co/xq2j32Jxao'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075699832285736960, 'text': 'Реально потрясающе. Путина спрашивают про свалки, а он: а вы про острова мусора в Тихом океане знаете?\n"Вы что, хот… https://t.co/MUsCpXCWQW'}
{'author_name': 'НТВ', 'tweet_id': 1075670731755806720, 'text': '71-летняя хабаровская пенсионерка принесла в полицию сверток с 250 тыс. рублей, который она нашла по дороге в магаз… https://t.co/Fln64Ejeqq'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075699832285736960, 'text': 'Реально потрясающе. Путина спрашивают про свалки, а он: а вы про острова мусора в Тихом океане знаете?\n"Вы что, хот… https://t.co/MUsCpXCWQW'}
{'author_name': 'РИА Новости', 'tweet_id': 1075752216458354688, 'text': 'Минкультуры: речи о возврате трофейных ценностей в Германию быть не может \nhttps://t.co/Mr3PRFZXKZ https://t.co/sKy2j4q64M'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075699832285736960, 'text': 'Реально потрясающе. Путина спрашивают про свалки, а он: а вы про острова мусора в Тихом океане знаете?\n"Вы что, хот… https://t.co/MUsCpXCWQW'}
{'author_name': 'РИА Новости', 'tweet_id': 1075779880988495874, 'text': 'Мадрид обвинил Пучдемона в растрате восьми миллионов евро на референдум \nhttps://t.co/XUKsKRIrft https://t.co/2YxhmdScUf'}
{'author_name': 'РИА Новости', 'tweet_id': 1075754825499975681, 'text': '#РИА_Видео\nРоссийский космонавт Сергей Прокопьев и астронавты Александр Герст и Серина Ауньон-Чэнселлор успешно вер… https://t.co/VXklyKCiJq'}
{'author_name': 'Афиша', 'tweet_id': 1075780984207896576, 'text': 'Оказалось, что сайты Госдумы, «Единой России» и издания «Вести.ру» используют для сбора данных «Яндекс.Метрику», но… https://t.co/lzOzNfCchW'}
{'author_name': 'ТАСС', 'tweet_id': 1075769165133168642, 'text': 'В России до 2024 года должны построить 200 мусороперерабатывающих заводов. По словам президента, ежегодно в стране… https://t.co/yqX8dmL4TJ'}
{'author_name': 'РЕН ТВ | Новости', 'tweet_id': 1075773168743178240, 'text': 'Падение империи: Начались слушания по иску пайщиков совхоза им Ленина к Грудинину https://t.co/ECPSi3PYT2 https://t.co/LIlmMCBX75'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075762690646102016, 'text': 'Очень круто. Помните такого жулика Серуканова -"очередного разочаровавшегося волонтёра Навального"? СМИ ещё обожали… https://t.co/vzCCVUgMO6'}
{'author_name': 'Интерфакс', 'tweet_id': 1075764009532444678, 'text': 'Пьяные украинцы пытались прорваться в Белоруссию https://t.co/RAeApkCjYH'}
{'author_name': 'НТВ', 'tweet_id': 1075670731755806720, 'text': '71-летняя хабаровская пенсионерка принесла в полицию сверток с 250 тыс. рублей, который она нашла по дороге в магаз… https://t.co/Fln64Ejeqq'}
{'author_name': 'Интерфакс', 'tweet_id': 1075504279358713858, 'text': 'Песков заявил о состоянии информационной войны в мире https://t.co/ssM6hU96Yu'}
{'author_name': 'Интерфакс', 'tweet_id': 1075774274961530880, 'text': 'Очевидцы сообщили о возгорании автомобиля в аэропорту "Внуково" https://t.co/xfpS2ngD9U'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075699832285736960, 'text': 'Реально потрясающе. Путина спрашивают про свалки, а он: а вы про острова мусора в Тихом океане знаете?\n"Вы что, хот… https://t.co/MUsCpXCWQW'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075707031707926528, 'text': 'На этой скучной, тоскливой пресс-конференции Путина точно есть как минимум один хороший журналист https://t.co/6CzQlERizA'}
{'author_name': 'РИА Новости', 'tweet_id': 1075777490428223489, 'text': 'Казанский авиазавод начал производство первых модернизированных Ту-160 \nhttps://t.co/BPZNcUGFVX https://t.co/FZCAOyYo8w'}
{'author_name': 'Навальный LIVE', 'tweet_id': 1075662493807079425, 'text': 'Сегодня Владимир Путин проводит свою очередную пресс-конференцию. Мы спросили у людей на улицах, какие вопросы они… https://t.co/5CtJ2pPvAd'}
{'author_name': 'Max', 'tweet_id': 1075756824400408577, 'text': 'ничего страшного! Это же не митинг @navalny с "вы мешаете проходу граждан" https://t.co/guF4TbAQYi'}
{'author_name': 'НТВ', 'tweet_id': 1075770598880555008, 'text': 'Путин снова исполнил мечту тяжелобольного ребенка. После большой пресс-конференции он побеседовал с 17-летней Регин… https://t.co/UC648J8OPy'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075699832285736960, 'text': 'Реально потрясающе. Путина спрашивают про свалки, а он: а вы про острова мусора в Тихом океане знаете?\n"Вы что, хот… https://t.co/MUsCpXCWQW'}
{'author_name': 'ТАСС', 'tweet_id': 1075769165133168642, 'text': 'В России до 2024 года должны построить 200 мусороперерабатывающих заводов. По словам президента, ежегодно в стране… https://t.co/yqX8dmL4TJ'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075762690646102016, 'text': 'Очень круто. Помните такого жулика Серуканова -"очередного разочаровавшегося волонтёра Навального"? СМИ ещё обожали… https://t.co/vzCCVUgMO6'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075699832285736960, 'text': 'Реально потрясающе. Путина спрашивают про свалки, а он: а вы про острова мусора в Тихом океане знаете?\n"Вы что, хот… https://t.co/MUsCpXCWQW'}
{'author_name': 'Первый канал', 'tweet_id': 1075763781462941702, 'text': 'https://t.co/WmcIN8ZWz4'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075699832285736960, 'text': 'Реально потрясающе. Путина спрашивают про свалки, а он: а вы про острова мусора в Тихом океане знаете?\n"Вы что, хот… https://t.co/MUsCpXCWQW'}
{'author_name': 'РЕН ТВ | Новости', 'tweet_id': 1075770497474805760, 'text': '"Тут не мойка, тварь": охранник закрыл дверь в травмпункт перед окровавленными девушками https://t.co/3g6fcFBlpO https://t.co/uXQK54zSR3'}
{'author_name': 'РИА Новости', 'tweet_id': 1075776133164019712, 'text': 'Курс доллара поднялся выше 68 рублей \nhttps://t.co/qZzT21xQr9 https://t.co/j09E9Vobxg'}
{'author_name': 'НТВ', 'tweet_id': 1075698405702291456, 'text': 'Выдержке Путина можно только позавидовать: украинский журналист ведет себя довольно хамски, предлагает сменить мест… https://t.co/mmc8lhhyJg'}
{'author_name': 'РИА Новости', 'tweet_id': 1075779226580639745, 'text': 'Путин о допинговых скандалах: в значительной степени мы сами виноваты \nhttps://t.co/tooroiOLqV https://t.co/7kX1VUez3D'}
{'author_name': 'Афиша', 'tweet_id': 1075780984207896576, 'text': 'Оказалось, что сайты Госдумы, «Единой России» и издания «Вести.ру» используют для сбора данных «Яндекс.Метрику», но… https://t.co/lzOzNfCchW'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075770316884824064, 'text': 'После прямой линии Путина о\xa0чем будем говорить на\xa0моей прямой линии? https://t.co/og2DuO69xu'}
{'author_name': 'Meduza in English', 'tweet_id': 1075756455943270401, 'text': "Russia's courts blocked @Navalny's new project to defeat United Russia because it allegedly violates data privacy l… https://t.co/YDkpIAi57Z"}
{'author_name': 'Max', 'tweet_id': 1075756824400408577, 'text': 'ничего страшного! Это же не митинг @navalny с "вы мешаете проходу граждан" https://t.co/guF4TbAQYi'}
{'author_name': 'ТАСС', 'tweet_id': 1075428203794194434, 'text': 'На юго-западе Китая построили необычный жилой комплекс. У каждого жильца высотки будет собственный сад за окном.\n\nВ… https://t.co/hfuQRQmIV7'}
{'author_name': 'Интерфакс', 'tweet_id': 1075780927677030401, 'text': 'Школьник умер на уроке физкультуры в Дербенте https://t.co/9hp4IYAI8Y'}
{'author_name': 'Афиша', 'tweet_id': 1075780984207896576, 'text': 'Оказалось, что сайты Госдумы, «Единой России» и издания «Вести.ру» используют для сбора данных «Яндекс.Метрику», но… https://t.co/lzOzNfCchW'}
{'author_name': 'Сергей Ежов', 'tweet_id': 1075709673716756480, 'text': 'Отлично работает ФСО. Лидера банды Цапка на инаугурацию президента допускала, а главреда одного из главных расследо… https://t.co/5sB1my5aaQ'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075699832285736960, 'text': 'Реально потрясающе. Путина спрашивают про свалки, а он: а вы про острова мусора в Тихом океане знаете?\n"Вы что, хот… https://t.co/MUsCpXCWQW'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075712448181780480, 'text': 'Самый смешной момент. Британский подданный Брилёв от имени российского телевидения спрашивает Путина об отношениях с Британией.'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075712448181780480, 'text': 'Самый смешной момент. Британский подданный Брилёв от имени российского телевидения спрашивает Путина об отношениях с Британией.'}
{'author_name': 'Vitaly Kolesnikov', 'tweet_id': 1075699468362747904, 'text': '— Владимир Владимирович, а что со свалками делать будет?\n—Вы знаете, в тихом океане есть целые острова мусора...'}
{'author_name': 'Интерфакс', 'tweet_id': 1075781880891359233, 'text': 'Судоходство в Керченском проливе будет восстановлено в четверг вечером https://t.co/25okAV8Bjf'}
{'author_name': 'ТАСС', 'tweet_id': 1075775659052478465, 'text': 'Владимир Путин убежден в правильности и неизбежности увеличения пенсионного возраста. Глава государства добавил, чт… https://t.co/ZeIFhuNd2S'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075712448181780480, 'text': 'Самый смешной момент. Британский подданный Брилёв от имени российского телевидения спрашивает Путина об отношениях с Британией.'}
{'author_name': 'RT на русском', 'tweet_id': 1075737439552106496, 'text': '«Это штамп»: Владимир Путин рассказал, хочет ли он править миром https://t.co/YDdAHFrbuK https://t.co/sqXtTQnlMV'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075712448181780480, 'text': 'Самый смешной момент. Британский подданный Брилёв от имени российского телевидения спрашивает Путина об отношениях с Британией.'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075770316884824064, 'text': 'После прямой линии Путина о\xa0чем будем говорить на\xa0моей прямой линии? https://t.co/og2DuO69xu'}
{'author_name': 'RT на русском', 'tweet_id': 1075771177128214532, 'text': 'В Киеве верующие провели акцию протеста у здания Рады\nhttps://t.co/VVTVSYhII3 https://t.co/yK2mReEGA1'}
{'author_name': 'Meduza in English', 'tweet_id': 1075756455943270401, 'text': "Russia's courts blocked @Navalny's new project to defeat United Russia because it allegedly violates data privacy l… https://t.co/YDkpIAi57Z"}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075699832285736960, 'text': 'Реально потрясающе. Путина спрашивают про свалки, а он: а вы про острова мусора в Тихом океане знаете?\n"Вы что, хот… https://t.co/MUsCpXCWQW'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075699832285736960, 'text': 'Реально потрясающе. Путина спрашивают про свалки, а он: а вы про острова мусора в Тихом океане знаете?\n"Вы что, хот… https://t.co/MUsCpXCWQW'}
{'author_name': 'РИА Новости', 'tweet_id': 1075776133164019712, 'text': 'Курс доллара поднялся выше 68 рублей \nhttps://t.co/qZzT21xQr9 https://t.co/j09E9Vobxg'}
{'author_name': 'Вести.Ru', 'tweet_id': 1075782466122657793, 'text': 'Путин поручил Бастрыкину лично проконтролировать расследование убийства братьев-пастухов https://t.co/lO1NV6sjzq'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075699832285736960, 'text': 'Реально потрясающе. Путина спрашивают про свалки, а он: а вы про острова мусора в Тихом океане знаете?\n"Вы что, хот… https://t.co/MUsCpXCWQW'}
{'author_name': 'Вести.Ru', 'tweet_id': 1075771644428206080, 'text': 'Первые 2,5 тысячи новых Калашниковых переданы армии https://t.co/yONmsPo72w'}
{'author_name': 'Вести.Ru', 'tweet_id': 1075047121911496705, 'text': 'Бывшему главе львовского "Правого сектора" перерезали горло https://t.co/oyjsXz1Nl4'}
{'author_name': 'ТАСС', 'tweet_id': 1075775659052478465, 'text': 'Владимир Путин убежден в правильности и неизбежности увеличения пенсионного возраста. Глава государства добавил, чт… https://t.co/ZeIFhuNd2S'}
{'author_name': 'ТАСС', 'tweet_id': 1075776973496934400, 'text': 'Курс доллара на Мосбирже превысил 68 рублей:\nhttps://t.co/R6FNW86yen https://t.co/D3h6Vffy4W'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075707031707926528, 'text': 'На этой скучной, тоскливой пресс-конференции Путина точно есть как минимум один хороший журналист https://t.co/6CzQlERizA'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075762690646102016, 'text': 'Очень круто. Помните такого жулика Серуканова -"очередного разочаровавшегося волонтёра Навального"? СМИ ещё обожали… https://t.co/vzCCVUgMO6'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075699832285736960, 'text': 'Реально потрясающе. Путина спрашивают про свалки, а он: а вы про острова мусора в Тихом океане знаете?\n"Вы что, хот… https://t.co/MUsCpXCWQW'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075762690646102016, 'text': 'Очень круто. Помните такого жулика Серуканова -"очередного разочаровавшегося волонтёра Навального"? СМИ ещё обожали… https://t.co/vzCCVUgMO6'}
{'author_name': 'Вести.Ru', 'tweet_id': 1075782212279185408, 'text': 'На въезде в аэропорт Внуково сгорела машина https://t.co/E5PSc9F8YS'}
{'author_name': 'Вести.Ru', 'tweet_id': 1075782212279185408, 'text': 'На въезде в аэропорт Внуково сгорела машина https://t.co/E5PSc9F8YS'}
{'author_name': 'ТАСС', 'tweet_id': 1075768332249251840, 'text': 'Газета The Washington Post узнала о намерениях США ввести новые санкции против Китая:\nhttps://t.co/9sjVnst6WD https://t.co/FOcgHXsYAF'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075699832285736960, 'text': 'Реально потрясающе. Путина спрашивают про свалки, а он: а вы про острова мусора в Тихом океане знаете?\n"Вы что, хот… https://t.co/MUsCpXCWQW'}
{'author_name': 'Интерфакс', 'tweet_id': 1075768541742194689, 'text': 'Китай провел испытание новой баллистической ракеты подводных лодок https://t.co/s3Ewt0kYfY'}
{'author_name': 'Интерфакс', 'tweet_id': 1075770979039604736, 'text': 'Возбуждено дело о хищениях при реконструкции военного авиазавода в Ульяновске https://t.co/UTiyFOLToq'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075699832285736960, 'text': 'Реально потрясающе. Путина спрашивают про свалки, а он: а вы про острова мусора в Тихом океане знаете?\n"Вы что, хот… https://t.co/MUsCpXCWQW'}
{'author_name': 'Афиша', 'tweet_id': 1075780984207896576, 'text': 'Оказалось, что сайты Госдумы, «Единой России» и издания «Вести.ру» используют для сбора данных «Яндекс.Метрику», но… https://t.co/lzOzNfCchW'}
{'author_name': 'Афиша', 'tweet_id': 1075780984207896576, 'text': 'Оказалось, что сайты Госдумы, «Единой России» и издания «Вести.ру» используют для сбора данных «Яндекс.Метрику», но… https://t.co/lzOzNfCchW'}
{'author_name': 'ТАСС', 'tweet_id': 1075768476105461760, 'text': 'Пробки в Москве достигли девяти баллов:\nhttps://t.co/roAH2hy4nr https://t.co/dRXYRXqU0m'}
{'author_name': 'Фёдор Крашенинников', 'tweet_id': 1075718978847293440, 'text': 'Путин бесконечно готов рассуждать о проблемах коренных парижан.'}
{'author_name': 'НТВ', 'tweet_id': 1075782704396943361, 'text': 'В Apple заявили, что кривой корпус iPad Pro – это нормально. \n\nПро изгибы на новых девайсах рассказали несколько бл… https://t.co/gGrj8ukALX'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075699832285736960, 'text': 'Реально потрясающе. Путина спрашивают про свалки, а он: а вы про острова мусора в Тихом океане знаете?\n"Вы что, хот… https://t.co/MUsCpXCWQW'}
{'author_name': 'НТВ', 'tweet_id': 1075670731755806720, 'text': '71-летняя хабаровская пенсионерка принесла в полицию сверток с 250 тыс. рублей, который она нашла по дороге в магаз… https://t.co/Fln64Ejeqq'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075712448181780480, 'text': 'Самый смешной момент. Британский подданный Брилёв от имени российского телевидения спрашивает Путина об отношениях с Британией.'}
{'author_name': 'ТАСС', 'tweet_id': 1075768966755147776, 'text': 'Путин надеется через СМИ донести свою позицию по важным вопросам до миллионов людей:\nhttps://t.co/JvHW2biKGF https://t.co/KilmSe5eQ7'}
{'author_name': 'ТАСС', 'tweet_id': 1075767846452355075, 'text': 'В Петербурге сошёл на воду уникальный ледокол, умеющий ломать двухметровые льды и обладающий повышенной маневреннос… https://t.co/yL1dikjDft'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075707031707926528, 'text': 'На этой скучной, тоскливой пресс-конференции Путина точно есть как минимум один хороший журналист https://t.co/6CzQlERizA'}
{'author_name': 'Вести.Ru', 'tweet_id': 1075730172928802816, 'text': '"Не дождетесь!" - Владимир Путин рассказал о своем здоровье. https://t.co/SWuIEjmfm4'}
{'author_name': 'Вести.Ru', 'tweet_id': 1075782466122657793, 'text': 'Путин поручил Бастрыкину лично проконтролировать расследование убийства братьев-пастухов https://t.co/lO1NV6sjzq'}
{'author_name': 'Афиша', 'tweet_id': 1075780984207896576, 'text': 'Оказалось, что сайты Госдумы, «Единой России» и издания «Вести.ру» используют для сбора данных «Яндекс.Метрику», но… https://t.co/lzOzNfCchW'}
{'author_name': 'Интерфакс', 'tweet_id': 1075768542048378880, 'text': 'Прокуратура попросила смягчить наказание осужденным бывшим топ-менеджерам "Ленэнерго" https://t.co/FOTkS8Y51O'}
{'author_name': 'Vitaly Kolesnikov', 'tweet_id': 1075699468362747904, 'text': '— Владимир Владимирович, а что со свалками делать будет?\n—Вы знаете, в тихом океане есть целые острова мусора...'}
{'author_name': 'РИА Новости', 'tweet_id': 1075782481759080448, 'text': 'Российские физики сделали титан неуязвимым для действия кислот \nhttps://t.co/nsaz5HDo40 https://t.co/2CsD6O4mAC'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075691448765427713, 'text': 'Путина спрашивают про общественные, культурные и прочие советы. Он начинает описывать подвиг десантников. Зал хлопает.\nПаноптикум какой-то.'}
{'author_name': 'НТВ', 'tweet_id': 1075670731755806720, 'text': '71-летняя хабаровская пенсионерка принесла в полицию сверток с 250 тыс. рублей, который она нашла по дороге в магаз… https://t.co/Fln64Ejeqq'}
{'author_name': 'НТВ', 'tweet_id': 1075670731755806720, 'text': '71-летняя хабаровская пенсионерка принесла в полицию сверток с 250 тыс. рублей, который она нашла по дороге в магаз… https://t.co/Fln64Ejeqq'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075699832285736960, 'text': 'Реально потрясающе. Путина спрашивают про свалки, а он: а вы про острова мусора в Тихом океане знаете?\n"Вы что, хот… https://t.co/MUsCpXCWQW'}
{'author_name': 'НТВ', 'tweet_id': 1075782766170726400, 'text': 'В Apple на это ответили, что это просто «побочный эффект производства», изгиб появляется из-за охлаждения металличе… https://t.co/nus75ns6rk'}
{'author_name': 'НТВ', 'tweet_id': 1075730751906344961, 'text': '— Как вы себя чувствуете?\n— Не дождетесь.\n#прессконференция https://t.co/U7HaeSKXau'}
{'author_name': 'ТАСС', 'tweet_id': 1075782697816129537, 'text': 'США рассчитывают, что новая встреча Дональда Трампа и Ким Чен Ына пройдет в начале 2019 года:… https://t.co/nMN7qB9FAf'}
{'author_name': 'НТВ', 'tweet_id': 1075782704396943361, 'text': 'В Apple заявили, что кривой корпус iPad Pro – это нормально. \n\nПро изгибы на новых девайсах рассказали несколько бл… https://t.co/gGrj8ukALX'}
{'author_name': 'Интерфакс', 'tweet_id': 1075776781993500674, 'text': 'Украинские корабли снова собрались пройти через Керченский пролив https://t.co/0VuIMYqpBd'}
{'author_name': 'ТАСС', 'tweet_id': 1075768101340213252, 'text': 'Владимир Путин ответил на вопрос о качестве контента на российских телеканалах. По его словам, он мало смотрит теле… https://t.co/KDoZdwA64o'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075712448181780480, 'text': 'Самый смешной момент. Британский подданный Брилёв от имени российского телевидения спрашивает Путина об отношениях с Британией.'}
{'author_name': 'НТВ', 'tweet_id': 1075776196384763904, 'text': 'В интервью 17-летней Регине Путин признался, что не воспринимает тяжелую музыку. «Heavy metal для меня очень сложно… https://t.co/s1s9QIk1ro'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075691448765427713, 'text': 'Путина спрашивают про общественные, культурные и прочие советы. Он начинает описывать подвиг десантников. Зал хлопает.\nПаноптикум какой-то.'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075699832285736960, 'text': 'Реально потрясающе. Путина спрашивают про свалки, а он: а вы про острова мусора в Тихом океане знаете?\n"Вы что, хот… https://t.co/MUsCpXCWQW'}
{'author_name': 'РИА Новости', 'tweet_id': 1075776951917281281, 'text': 'В Москве из желудка двухлетней девочки извлекли пятисантиметровый гвоздь https://t.co/pf2fEuiWIH'}
{'author_name': 'РЕН ТВ | Новости', 'tweet_id': 1075777405359255553, 'text': 'В Херсоне чиновник избил пенсионера и женщину в рабочем кабинете — #ВИДЕО https://t.co/IPtCRuz5zF https://t.co/0oSgnlN6HT'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075684302313189376, 'text': 'Лучшая прямая трансляция пресс-конференции Путина (с комментариями!) здесь: https://t.co/Mp4iObvBzE'}
{'author_name': 'ТАСС', 'tweet_id': 1075782697816129537, 'text': 'США рассчитывают, что новая встреча Дональда Трампа и Ким Чен Ына пройдет в начале 2019 года:… https://t.co/nMN7qB9FAf'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075699832285736960, 'text': 'Реально потрясающе. Путина спрашивают про свалки, а он: а вы про острова мусора в Тихом океане знаете?\n"Вы что, хот… https://t.co/MUsCpXCWQW'}
{'author_name': 'ТАСС', 'tweet_id': 1075777338854432777, 'text': 'Де Мистура назвал конституционный комитет не единственной базой для урегулирования в Сирии:\nhttps://t.co/Jr247JpUrA https://t.co/wUYS5rDo6U'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075741456336347136, 'text': 'Как наш Дима Низовцев  @zimbru_khv ходил на пресс-конференцию Путина и как ФСО у него плакаты отбирала. Посмотрите,… https://t.co/1IylYx45VQ'}
{'author_name': 'РЕН ТВ | Новости', 'tweet_id': 1075777405359255553, 'text': 'В Херсоне чиновник избил пенсионера и женщину в рабочем кабинете — #ВИДЕО https://t.co/IPtCRuz5zF https://t.co/0oSgnlN6HT'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075699832285736960, 'text': 'Реально потрясающе. Путина спрашивают про свалки, а он: а вы про острова мусора в Тихом океане знаете?\n"Вы что, хот… https://t.co/MUsCpXCWQW'}
{'author_name': 'ТАСС', 'tweet_id': 1075782841714266112, 'text': 'В ООН усомнились в легитимности некоторых кандидатов в конституционный комитет Сирии:\nhttps://t.co/x09EIR1XIF https://t.co/pjWQ4cShdU'}
{'author_name': 'РИА Новости', 'tweet_id': 1075777490428223489, 'text': 'Казанский авиазавод начал производство первых модернизированных Ту-160 \nhttps://t.co/BPZNcUGFVX https://t.co/FZCAOyYo8w'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075770316884824064, 'text': 'После прямой линии Путина о\xa0чем будем говорить на\xa0моей прямой линии? https://t.co/og2DuO69xu'}
{'author_name': 'НТВ', 'tweet_id': 1075770598880555008, 'text': 'Путин снова исполнил мечту тяжелобольного ребенка. После большой пресс-конференции он побеседовал с 17-летней Регин… https://t.co/UC648J8OPy'}
{'author_name': 'ТАСС', 'tweet_id': 1075782928062443527, 'text': '#ТАСС_Инфографика\nПомните, в прошлую новогоднюю ночь в Южно-Сахалинске дотла сгорела главная елка города. Причина —… https://t.co/xY6K4fe927'}
{'author_name': 'ТАСС', 'tweet_id': 1075428203794194434, 'text': 'На юго-западе Китая построили необычный жилой комплекс. У каждого жильца высотки будет собственный сад за окном.\n\nВ… https://t.co/hfuQRQmIV7'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075722851817324546, 'text': 'Напоминаю, что наша трансляция с комментариями здесь. 14 тысяч человек смотрят в онлайне https://t.co/Mp4iObvBzE'}
{'author_name': 'ТАСС', 'tweet_id': 1075777338854432777, 'text': 'Де Мистура назвал конституционный комитет не единственной базой для урегулирования в Сирии:\nhttps://t.co/Jr247JpUrA https://t.co/wUYS5rDo6U'}
{'author_name': 'РИА Новости', 'tweet_id': 1075776133164019712, 'text': 'Курс доллара поднялся выше 68 рублей \nhttps://t.co/qZzT21xQr9 https://t.co/j09E9Vobxg'}
{'author_name': 'Проф. Преображенский', 'tweet_id': 1075717989180338176, 'text': 'Журналист:\n- Давайте отменим повышение пенсионного возраста!\nПутин: \n- Нет, сейчас страна развивается, крепнет, поэ… https://t.co/DjRHtml6nQ'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075770316884824064, 'text': 'После прямой линии Путина о\xa0чем будем говорить на\xa0моей прямой линии? https://t.co/og2DuO69xu'}
{'author_name': 'RT на русском', 'tweet_id': 1075757544990277632, 'text': 'В Британии очередной скандал: все читают Корбина по губам. Во время речи Терезы Мэй в парламенте он прошептал что-т… https://t.co/wA2q9dXjfl'}
{'author_name': 'РИА Новости', 'tweet_id': 1075691602536992768, 'text': 'Путин: "По поводу задержания рэперов - я с вами согласен. Это результат обратный ожиданиям. Это ни к чему не ведет.… https://t.co/UGYV1dw1wv'}
{'author_name': 'РИА Новости', 'tweet_id': 1075783070945550337, 'text': 'Симоньян дала наставление девушке перед интервью с Путиным \nhttps://t.co/v81DlIMLmi https://t.co/386r6xT5BQ'}
{'author_name': 'РИА Новости', 'tweet_id': 1075768880008617985, 'text': 'Трамп с опечатками рассказал о проделанной за Россию работу в Сирии \nhttps://t.co/ntFAhetC6J https://t.co/QDJRXgp1Tl'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075341550429835265, 'text': 'И нам очень нужны крутые разработчики, желающие сделать доброе дело и нахлобучить «Единую Россию», Роскомнадзор и в… https://t.co/i9Avh57lv0'}
{'author_name': 'ТАСС', 'tweet_id': 1075768966755147776, 'text': 'Путин надеется через СМИ донести свою позицию по важным вопросам до миллионов людей:\nhttps://t.co/JvHW2biKGF https://t.co/KilmSe5eQ7'}
{'author_name': 'ТАСС', 'tweet_id': 1075776973496934400, 'text': 'Курс доллара на Мосбирже превысил 68 рублей:\nhttps://t.co/R6FNW86yen https://t.co/D3h6Vffy4W'}
{'author_name': 'Афиша', 'tweet_id': 1075780984207896576, 'text': 'Оказалось, что сайты Госдумы, «Единой России» и издания «Вести.ру» используют для сбора данных «Яндекс.Метрику», но… https://t.co/lzOzNfCchW'}
{'author_name': 'РИА Новости', 'tweet_id': 1075754825499975681, 'text': '#РИА_Видео\nРоссийский космонавт Сергей Прокопьев и астронавты Александр Герст и Серина Ауньон-Чэнселлор успешно вер… https://t.co/VXklyKCiJq'}
{'author_name': 'Alexey Navalny', 'tweet_id': 1075712448181780480, 'text': 'Самый смешной момент. Британский подданный Брилёв от имени российского телевидения спрашивает Путина об отношениях с Британией.'}
{'author_name': 'РИА Новости', 'tweet_id': 1075782481759080448, 'text': 'Российские физики сделали титан неуязвимым для действия кислот \nhttps://t.co/nsaz5HDo40 https://t.co/2CsD6O4mAC'}
